In [5]:
#### ASSOCIATED DATA CAN BE FOUND AT ZENODO REPO FROM https://doi.org/10.1186/s13059-023-02846-8 ####

import numpy as np
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
from scipy import stats
import re
from scipy.stats.stats import pearsonr
#import rpy2.robjects as robjects
import random
from statsmodels.stats.multitest import fdrcorrection
import os
import seaborn as sns
#import gseapy as gs
from scipy.stats import norm
import gseapy as gs
from scipy.stats import binom_test
import copy

min_gen = 6
max_gen = 300

#List of imprinted genes was obtained from https://www.geneimprint.com/site/genes-by-species
imp_conf_list = []
o = open("Imprinted_Genes.txt")
for line in o:
    l = line.split("\t")
    if "Imprinted" in l[1]:
        imp_conf_list.append(l[0])

C:\Users\Alex\AppData\Local\Temp/ipykernel_42460/2789269303.py:8: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [2]:
#Read in HPO.
HPO = pd.read_csv("HPO_World_14_Typ.csv")
HPO = HPO.set_index("HPO_IDs")
HPO_Conv = pd.read_csv("HPO_Conversion.csv")
Convert_HPO = {}
gene_set_hpo_brain = []
gene_set_hpo_skel = []
for index, row in HPO_Conv.iterrows():
    Convert_HPO[row[0]] = row[1]
HPO.head()

d_HPO = {}
for index, row in HPO.iterrows():
    x = Convert_HPO[index]
    d_HPO[x] = []
    for i in zip(row.index.tolist(), list(row)):
        if i[1]:
            d_HPO[Convert_HPO[index]].append(i[0])

phen = pd.read_csv("specific_phenotype_ontology_v14.csv")
keeper1 = "brain"
keep_brain = []
keeper2 = "skel"
keep_skel = []
keeper3 = "heart"
keep_heart = []
keeper4 = "eye"
keep_eye = []
keeper5 = "skeletal muscle" #in system
keep_musc = []
keeper6 = "pancreas"
keep_panc = []
keeper7 = "liver"
keep_liver = []
keeper8 = "peripheral ner"
keep_per = []
for index, row in phen.iterrows():
    organ = row[4]
    system = row[3]
    try:
        if keeper1 in organ:
            keep_brain.append(row[1])
        if keeper3 in organ:
            keep_heart.append(row[1])
        if keeper4 in organ:
            keep_eye.append(row[1])
        if keeper6 in organ:
            keep_panc.append(row[1])
        if keeper7 in organ:
            keep_liver.append(row[1])
        if keeper8 in organ:
            keep_per.append(row[1])
        
    except:
        pass
    try:
        if keeper2 in system:
            keep_skel.append(row[1])
        if keeper5 in system:
            keep_musc.append(row[1])
    except:
        pass
d_HPO_brain = {}
d_HPO_skel = {}
d_HPO_heart = {}
d_HPO_eye = {}
d_HPO_musc = {}
d_HPO_panc = {}
d_HPO_liver = {}
d_HPO_per = {}
for key in d_HPO.keys():
    if key in keep_brain:
        d_HPO_brain[key] = d_HPO[key]
    if key in keep_skel:
        d_HPO_skel[key] = d_HPO[key]
    if key in keep_heart:
        d_HPO_heart[key] = d_HPO[key]
    if key in keep_eye:
        d_HPO_eye[key] = d_HPO[key]
    if key in keep_musc:
        d_HPO_musc[key] = d_HPO[key]
    if key in keep_panc:
        d_HPO_panc[key] = d_HPO[key]
    if key in keep_liver:
        d_HPO_liver[key] = d_HPO[key]
    if key in keep_per:
        d_HPO_per[key] = d_HPO[key]
    
        


In [3]:
#Read in Gene Ontology
d_CC = {}
d_BP = {}
d_MF = {}
gene_set_cc = []
gene_set_bp = []
gene_set_mf = []
CC = pd.read_csv("OFFICIAL_GENE_SYMBOL2GOTERM_CC_DIRECT.txt", sep = "\t", header = None)
for index, row in CC.iterrows():
    if row[1] not in d_CC.keys():
        d_CC[row[1]] = [row[0]]
    else:
        d_CC[row[1]].append(row[0])
        
BP = pd.read_csv("OFFICIAL_GENE_SYMBOL2GOTERM_BP_DIRECT.txt", sep = "\t", header = None)
for index, row in BP.iterrows():
    if row[1] not in d_BP.keys():
        d_BP[row[1]] = [row[0]]
    else:
        d_BP[row[1]].append(row[0])
        
MF = pd.read_csv("OFFICIAL_GENE_SYMBOL2GOTERM_MF_DIRECT.txt", sep = "\t", header = None)
for index, row in MF.iterrows():
    if row[1] not in d_MF.keys():
        d_MF[row[1]] = [row[0]]
    else:
        d_MF[row[1]].append(row[0])

for key in d_MF.keys():
    z = d_MF[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_mf = gene_set_mf + z
for key in d_CC.keys():
    z = d_CC[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_cc = gene_set_cc + z
for key in d_BP.keys():
    z = d_BP[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_bp = gene_set_bp + z
gene_set_cc = list(set(gene_set_cc))
gene_set_bp = list(set(gene_set_bp))
gene_set_mf = list(set(gene_set_mf))

In [4]:
#Read in GAD
d_GAD = {}
d_GADC = {}
gene_set_gad = []
GAD = pd.read_csv("OFFICIAL_GENE_SYMBOL2GAD_DISEASE.txt", sep = "\t", header = None)
GADC = pd.read_csv("OFFICIAL_GENE_SYMBOL2GAD_DISEASE_CLASS.txt", sep = "\t", header = None)
for index, row in GAD.iterrows():
    if row[1] not in d_GAD.keys():
        d_GAD[row[1]] = [row[0]]
    else:
        d_GAD[row[1]].append(row[0])
for index, row in GADC.iterrows():
    if row[1] not in d_GADC.keys():
        d_GADC[row[1]] = [row[0]]
    else:
        d_GADC[row[1]].append(row[0])
gene_set_GAD = []
for key in d_GAD.keys():
    z = d_GAD[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_GAD = gene_set_GAD + z
gene_set_GAD = list(set(gene_set_gad))

In [13]:
#Read in KEGG
d_KEGG = {}
gene_set_kegg = []
KEGG = pd.read_csv("KEGG_pathway_NEW.txt", sep = "\t")
for index, row in KEGG.iterrows():
    if row["pathway name"] not in d_KEGG.keys():
        d_KEGG[row["pathway name"]] = [row["gene symbol"]]
    else:
        d_KEGG[row["pathway name"]].append(row["gene symbol"])
        
for key in d_KEGG.keys():
    z = d_KEGG[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_kegg = gene_set_kegg + z
gene_set_kegg = list(set(gene_set_kegg))

In [6]:
#Finally read in REACTOME
d_REACT = {}
gene_set_react = []
REACT = pd.read_csv("OFFICIAL_GENE_SYMBOL2REACTOME_PATHWAY.txt", sep = "\t", header = None)
for index, row in REACT.iterrows():
    gene_set_react.append(row[0])
    if row[1] not in d_REACT:
        d_REACT[row[1]] = [row[0]]
    else:
        d_REACT[row[1]].append(row[0])
gene_set_react = list(set(gene_set_react))

In [8]:
cell_types = ["CM", "RPE", "MN", "SKM", "HP", "PP"]
dd = {}
for ct in cell_types:
    dd[ct] = list(pd.read_csv(ct + "_Filtered.txt", sep = "\t")["gene"])

In [24]:
suffix = "SKM_MinReads_Per_Allele10MWU_ASE_Ratios_Muscle"
v = pd.read_csv(suffix + ".txt", sep = "\t")
v = v[~v["gene"].isin(imp_conf_list)]
v = v[v["Sample Size"] > 20]
v.to_csv(suffix + "_Filt20.txt", sep = "\t", index = False)

In [ ]:
#Extracts gene info.
def extract_gene_info(genes, cell):
    name = "syntaxin_binding"
    ase = pd.read_csv(cell + "_Filtered.txt", sep = "\t")
    scores = pd.read_csv(cell + "_Ranks_Specific_Signed.csv")
    ase = ase[ase["gene"].isin(genes.split(";"))].set_index("gene").drop(["pvalue Humreffed", "padj Humreffed", "qval_mine Humreffed", "pvalue Chpreffed", "padj Chpreffed", "qval_mine Chpreffed"], axis = 1)
    scores = scores[scores["gene"].isin(genes.split(";"))].set_index("gene")
    out = []
    s = scores.join(ase)
    s.to_csv("Ledge_genes_info_" + name + "_" + cell + ".csv")
extract_gene_info("BLOC1S6;STX6;TXLNA;RAB4A;BET1;HECTD3;SNAP23;NAPA;SYT11;STXBP5;PTPN2;STXBP4;CACNA1A;SYT7;SNAP29;SYT4;SYT12;SYT13;STXBP3;ABCC8;SYTL3;SNAP47;SYT2;SYT6;SYTL2;C2CD4C", "MN")

In [ ]:
#Split by in up in human or down in human and rank for signed GSEA.
mapping = {"CM": "Heart", "SKM":"Muscle", "RPE":"All_Tissues", "PP":"Pancreas", "HP":"Liver", "MN":"Brain_Nerve"}
for key in mapping.keys():
    try:
        llll = mapping[key]
        s = pd.read_csv(key + "_Filtered.txt", sep = "\t")
        down = []
        up = []
        for index, row in s.iterrows():
            if (row["L2FC Humreffed"] + row["L2FC Chpreffed"])/2 < 0:
                down.append(row[0])
            elif (row["L2FC Humreffed"] + row["L2FC Chpreffed"])/2 > 0:
                up.append(row[0])

        r = pd.read_csv(key + "_MinReads_Per_Allele10MWU_ASE_Ratios_" + llll + "_Filt20.txt", sep = "\t")
        out_up = []
        out_down = []
        for index, row in r.iterrows():
            if row["gene"] in up:
                out_up.append(list(row))
            elif row["gene"] in down:
                out_down.append(list(row))
        df_up = pd.DataFrame(out_up)
        df_up.columns = r.columns
        df_up = df_up.sort_values("pval", ascending = False)
        df_up["MWU p-val Rank"] = range(1, len(list(df_up.index)) + 1)
        #df_up.to_csv(cell + "_Ranks_" + d + "_Up_Human.csv", index = False)
        df_down = pd.DataFrame(out_down)
        df_down.columns = r.columns
        df_down = df_down.sort_values("pval", ascending = False)
        df_down["MWU p-val Rank"] = range(1, len(list(df_down.index)) + 1)
        #df_down.to_csv(cell + "_Ranks_" + d + "_Down_Human.csv", index = False)


        v_up = df_up
        v_up = v_up.set_index("gene")
        v_down = df_down
        v_down = v_down.set_index("gene")
        p = pd.DataFrame()
        v_down["MWU p-val Rank"] = -1*v_down["MWU p-val Rank"]
        v = pd.concat([v_up, v_down]).sort_values("MWU p-val Rank", ascending = False)
        v.to_csv(key + "_Ranks_Specific_Signed.csv")
    except:
        pass

In [ ]:
mapping_hpo = {"CM": d_HPO_heart, "SKM":d_HPO_skel, "RPE":d_HPO_eye, "PP":d_HPO_panc, "HP":d_HPO_liver, "MN":d_HPO_per}
onts = {"KEGG":d_KEGG, "BP":d_BP, "CC":d_CC, "MF":d_MF, "HPO":"blank", "REACT":d_REACT}
files = os.listdir()
count = 0
for file in files:
    if "Ranks_Specific_Signed" in file and "CM" not in file:
        ct = file.split("_")[0]
        v = pd.read_csv(file, sep = ",")
        ranking = v[["gene", "MWU p-val Rank"]]
        for j in onts.keys():
            data = onts[j]
            if j == "KEGG":
                out_name_1 = ct + "_pval_ranks_gsea_Enrich_KEGG_Specific_Signed"
            elif j == "CC":
                out_name_1 = ct + "_pval_ranks_gsea_Enrich_GOCC_Specific_Signed"
            elif j == "BP":
                out_name_1 = ct + "_pval_ranks_gsea_Enrich_GOBP_Specific_Signed"
            elif j == "MF":
                out_name_1 = ct + "_pval_ranks_gsea_Enrich_GOMF_Specific_Signed"
            elif j == "REACT":
                out_name_1 = ct + "_pval_ranks_gsea_Enrich_REACT_Specific_Signed"
            elif j == "HPO":
                out_name_1 = ct + "_pval_ranks_gsea_Enrich_HPO_Specific_Signed"
                data = mapping_hpo[ct]
            out_name = out_name_1 + "_Specific.csv"
            print(ct, j)
            gs.prerank(rnk=ranking, gene_sets=data, processes=4, permutation_num=1000, outdir= 'C:/Users/Alex/Myriad_RNA/Expression/' + out_name_1, format='png', seed=6, min_size = 10, max_size = 300)
            count += 1

In [ ]:
folders = os.listdir()
mapping = {"CM": [], "SKM":[], "RPE":[], "PP":[], "HP":[], "MN":[]}
for folder in folders:
    if "Enrich" in folder and "Ban" not in folder and "Specific_Signed" in folder:
        print(folder)
        ct = folder.split("_")[0]
        db = folder.split("_")[5]
        v = pd.read_csv(folder + "/gseapy.prerank.gene_sets.report.csv")
        for index, row in v.iterrows():
            if row["fdr"] < 0.25:
                mapping[ct].append([ct, db, "Specific"] + list(row))
for key in mapping:
    l = pd.DataFrame(mapping[key])
    
    l.columns = ["Cell Type", "Ontology", "All Tissues Or Specific"] + list(v.columns)
    l = l[l["es"] > 0].sort_values("fdr")
    l.to_csv(key + "_GSEA_Specific_FDR25.csv", index = False, sep = ",")

In [ ]:
#Compute the binomial test on all genes in a certain category (folder is fi)
#Need to use shell script to move all the GSEA results to that folder.
#Sort ascending or descending depending on whether it is an up one or a down one.
#Extract the position of the least significant gene.
#Compare total to the non-total with binomial test, where non-total is direcitonality match in ledge genes and total is at the rank of least significant ledge gene.
#Do with two pass. 
def extract_gene_info(genes, genes_all, cell, pos):
    scores = pd.read_csv(cell + "_Ranks_Specific_Signed.csv")
    ase = pd.read_csv(cell + "_Filtered.txt", sep = "\t").set_index("gene")
    scores = scores.set_index("gene")
    c_neg = 0
    tot = 0
    out = []
    scores = scores.sort_values("MWU p-val Rank", ascending = not pos)
    rank = 0
    l = []
    c_pos = 0
    genes_keep = genes
    for gene, row in scores.iterrows():
        if gene in genes:
            score = list(row)
            score[0] = (ase.loc[gene]["L2FC Humreffed"] + ase.loc[gene]["L2FC Chpreffed"])/2
            out.append([gene] + score)
            if (ase.loc[gene]["L2FC Humreffed"] + ase.loc[gene]["L2FC Chpreffed"])/2 < 0:
                c_neg += 1
            elif (ase.loc[gene]["L2FC Humreffed"] + ase.loc[gene]["L2FC Chpreffed"])/2 > 0:
                c_pos += 1
            rank = scores.loc[gene]["MWU p-val Rank"]
    scores = scores.sort_values("MWU p-val Rank", ascending = pos)
    for gene, row in scores.iterrows():
        r = row["MWU p-val Rank"]
        if abs(r) < abs(rank):
            #print(rank, r)
            break
        else:
            if gene in genes_all and gene not in genes:
                score = list(row)
                score[0] = (ase.loc[gene]["L2FC Humreffed"] + ase.loc[gene]["L2FC Chpreffed"])/2
                out.append([gene] + score)
                if (ase.loc[gene]["L2FC Humreffed"] + ase.loc[gene]["L2FC Chpreffed"])/2 < 0:
                    c_neg += 1
                elif (ase.loc[gene]["L2FC Humreffed"] + ase.loc[gene]["L2FC Chpreffed"])/2 > 0:
                    c_pos += 1
                genes_keep = genes_keep + [gene]
    assert(len(list(set(l) & set(genes))) == 0)
    print(";".join(genes_keep))
    df = pd.DataFrame(out)
    l = list(scores.columns)
    l[0] = "L2FC signed"
    df.columns = ["Gene"] + l
    #df.to_csv("Ledge_genes_info_" + name + "_" + cell + "_" + data + "_" + comp + ".csv", index = False)
    return stats.binom_test(c_neg, c_neg + c_pos, 0.5), c_neg, c_neg + c_pos

f = os.listdir()
o = []
for folder in f:
    if "Specific_Signed" in folder and "ranks_gsea_Enrich" in folder:
        c = folder.split("_")[0]
        d = folder.split("_")[5]
        v = pd.read_csv(os.path.join(folder, "gseapy.prerank.gene_sets.report.csv"))
        for index, row in v.iterrows():
            if row["fdr"] < 0.25 and len(row["ledge_genes"].split(";")) > 3:
                print(list(row))
                r = list(row)
                r = [folder] + r
                if row["es"] < 0:
                    q = extract_gene_info(row["ledge_genes"].split(";"), row["genes"].split(";"), c, False)
                else:
                    q = extract_gene_info(row["ledge_genes"].split(";"), row["genes"].split(";"), c, True)
                r = r + list(q)
                o.append(r)
df = pd.DataFrame(o)
df.columns = ["File", "Term", "ES", "NES", "p-val", "FDR", "Total Genes in Geneset", "Intersect with our Gene Set", "Genes", "ledge genes", "Binomial p-val", "Down in Human", "Total"]

#Finish with fdr correction after filtering out categories that appear multiple times to avoid testing the same hypothesis.
from statsmodels.stats.multitest import fdrcorrection
d = {}
for index, row in df.iterrows():
    key = row["Term"]
    val = row
    if key in d.keys():
        d[key].append(val)
    else:
        d[key] = [val]
o = []
for key in d.keys():
    l = d[key]
    m = 100000
    m_pos = 0
    if len(l) > 1:
        for i in range(len(l)):
            j = l[i]
            if i == 0:
                m = j["FDR"]
            else:
                if j["FDR"] < m:
                    m = j["FDR"]
                    m_pos = i
        o.append(l[m_pos])
    else:
        o.append(l[0])
df = pd.DataFrame(o)
to_cor = list(df["Binomial p-val"])
corred = fdrcorrection(to_cor)
df["FDR Sign Test"] = corred[1]

df.to_csv("AGSEA_MWU_p-val_SignTest_Specific_025.csv", index = False)